# Tracking FII flows

Plots the Foreign Institutional Investment flows in/out of Indian capital markets.
Follows the documentation [here](https://github.com/shyams80/plutons/blob/master/docs-R/InvestmentFlowsIndia.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
source("config.R")
source("goofy/plot.common.R")

#initialize
iflows <- InvestmentFlowsIndia()

In [ ]:
fiiCash <- iflows$FiiCashMarket() %>%
    group_by(TIME_STAMP, SECURITY_TYPE) %>%
    summarize(BUY = sum(BUY_VALUE),
            SELL = sum(SELL_VALUE),
            NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    arrange(TIME_STAMP) %>%
    select(TIME_STAMP, SECURITY_TYPE, NET) %>%
    collect()

fiiCash %>% print(n = 5)

fiiCash %>% ungroup(TIME_STAMP) %>% top_n(n = 5, wt = TIME_STAMP) %>% print()

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)